In [1]:
from pyspark.sql import SparkSession


In [2]:
spark=SparkSession.builder.appName('practise').getOrCreate()

In [4]:
df_spark=spark.read.csv('/media/mayank/New Volume/datasets/tips.csv',header=True,inferSchema=True)

In [5]:
df_spark.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [6]:
df_spark.columns

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [7]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer=StringIndexer(inputCols=['sex','smoker','day','time'],
                      outputCols=['sex_indexed','smoker_indexed','day_indexed','time_indexed'])
df_r=indexer.fit(df_spark).transform(df_spark)

In [11]:
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [12]:
df_r.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [13]:
from pyspark.ml.feature import VectorAssembler

In [16]:
feature_assembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],
                                 outputCol='Independent Features')
output=feature_assembler.transform(df_r)

In [17]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [18]:
output.columns

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed',
 'Independent Features']

In [19]:
finalized_data=output.select('Independent Features','total_bill')

In [20]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [21]:
## training & testing spliting
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

In [22]:
from pyspark.ml.regression import LinearRegression

In [24]:
regressor=LinearRegression(featuresCol='Independent Features',labelCol='total_bill')


In [25]:
regressor=regressor.fit(train_data)

In [27]:
regressor.coefficients

DenseVector([3.1517, 3.3043, -0.9634, 2.2221, 0.3658, -2.2567])

In [28]:
regressor.intercept

1.4436816874421496

In [29]:
pred_result=regressor.evaluate(test_data)

In [30]:
pred_result.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.97,2.0])|     12.02|14.261031340518294|
|(6,[0,1],[2.31,3.0])|     18.69|18.636871419881697|
|(6,[0,1],[3.35,3.0])|     20.65|21.914620469589693|
| (6,[0,1],[3.6,3.0])|     24.06| 22.70254091423104|
|(6,[0,1],[3.76,2.0])|     18.24| 19.90254172415033|
|[1.0,2.0,1.0,1.0,...|      5.75|13.560113207726493|
|[1.01,2.0,1.0,0.0...|     16.99|  10.6378808233262|
|[1.17,2.0,0.0,1.0...|     32.83| 13.96177887298141|
|[1.36,3.0,1.0,0.0...|     18.64|13.154383448744936|
|[1.5,2.0,1.0,0.0,...|     10.65|10.291350623092917|
|[1.5,2.0,1.0,0.0,...|     11.17|10.291350623092917|
|[1.57,2.0,0.0,0.0...|     15.42|13.366186752527403|
|[1.66,3.0,0.0,0.0...|     10.34|16.954106387249457|
|[1.67,3.0,1.0,0.0...|     10.33|16.022259071830522|
|[1.68,2.0,1.0,0.0...|     13.42|10.858653343234687|
|[1.76,2.0,0.0,1.0...|     11.24|15.8212711223

In [31]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError,pred_result.r2

(4.39693705298286, 39.21692816798214, 0.5284505882015866)